<a href="https://colab.research.google.com/github/abhilashhn1993/word_count_with_mapreduce/blob/master/MapReduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import threading
import queue
import pandas as pd

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Input.txt to Input.txt


In [32]:
#Implementing methods to clean function
def cleanText(file):
   words = []
   reg = re.compile("[a-z]+")
   for line in file:
     if len(line) > 0:
       words.append(reg.findall(line.lower()))
   words = [i for i in words if i!=[]]
   return words

In [6]:
#Custom function to unlist the nested list of words
def unlist(input):
  list = []
  for i in input:
    for word in i:
      list.append(word)
  return list

In [7]:
#Data split function to divide the input data file into two halves
#First part includes the first 5000 lines of the text file and part 2 includes the rest
def split(token):
  words = []
  for word in token:
    words.append(word)
  file1 = unlist(words[0:5000])
  file2 = unlist(words[5001:])
  files = [file1, file2]
  return files

In [8]:
#Mapper function
def mapper1(list, mapper_q1):
  mapping = []
  for word in list:
    mapping.append("%s%s%d"%(word,',',1))
  #Adding the mapped word list into the queue using put method
  mapper_q1.put(mapping)

In [9]:
#Second mapper function for multithreading purposes (Same as above)
def mapper2(list, mapper_q2):
  mapping = []
  for word in list:
    mapping.append("%s%s%d"%(word,',', 1))
  #Adding the mapped word list into the queue using put method
  mapper_q2.put(mapping)

In [10]:
#Function to sort the mapped words
def sort_func(map1, map2):
  map = map1 + map2
  map.sort()
  return map

In [11]:
#Partition function to divide the tokens into two halves
# a-m words and n-z words
def partition(list):
  file1 = []
  file2 = []
  for item in list:
    val = re.search("^[a-m]", item) #Picking all the words starting from 'a-m'
    if val!= None:
      file1.append(item)
    else:
      file2.append(item)
  file = [file1,file2]
  return file

In [12]:
#Reducer function to collect all values belonging to the key and count the word frequencies
def reducer1(sortedList, reducer_q1):
  word_count = {}
  item = None
  current_count = 0
  for words in sortedList:
    word,count = words.split(',',1) #Splitting the element of list with ',' delimeter
    count = int(count)
    
    if item == word:
      current_count += count
    else:
      item = word
      current_count = count
    word_count[item] = current_count #Adding the Word and count to dictionary
  if item == word: #For the final word in the file
    word_count[item] = current_count
  #Returning the queue where the mapped dictionary is pushed
  return reducer_q1.put(word_count)

In [13]:
#Second reducer function for multithreading purposes (Same as above)
def reducer2(sortedList, reducer_q2):
  word_count = {}
  item = None
  current_count = 0
  for words in sortedList:
    word,count = words.split(',',1)
    count = int(count)
    
    if item == word:
      current_count += count
    else:
      item = word
      current_count = count
    word_count[item] = current_count
  if item == word:
    word_count[item] = current_count
  return reducer_q2.put(word_count)

In [27]:
#Main method calling the MapReduce function
def MapReduce(file):
  mapper_q1 = queue.Queue()
  mapper_q2 = queue.Queue()
  reducer_q1 = queue.Queue()
  reducer_q2 = queue.Queue()

  #Reading the Input file
  file = open(path, encoding="utf8")

  #Generating word Tokens after cleaning the text file
  wordTokens = cleanText(file)
  #Split the input file into two halves and return a list of two files
  filesList = split(wordTokens)

  #Creating threads to implement mapper functions
  mapping1 = threading.Thread(target=mapper1, args=(filesList[0], mapper_q1))
  mapping2 = threading.Thread(target=mapper2, args=(filesList[1], mapper_q2))
  mapping1.start()
  mapping2.start()
  mapping1.join()
  mapping2.join()
 
  #Retreiving the mapped elements from the queue into a list
  mapping1 =[]
  for element in mapper_q1.get():
    mapping1.append(element)
  
  #Retreiving the mapped elements from the queue into a list
  mapping2 =[]
  for element in mapper_q2.get():
    mapping2.append(element)

  #Implementing sorting and partition function  
  sortedMap = sort_func(mapping1, mapping2)
  sortedFile = partition(sortedMap)

  #Creating threads to implement reducer functions
  red1 = threading.Thread(target=reducer1, args = (sortedFile[0], reducer_q1))
  red2 = threading.Thread(target=reducer2, args = (sortedFile[1], reducer_q2))

  red1.start()
  red2.start()
  red1.join()
  red2.join()

  #Retreiving reduced elements from the queue elements
  red1 = reducer_q1.get()
  red2 = reducer_q2.get()

  #Reducer output converted into a csv file
  red1.update(red2)
  output_file = pd.DataFrame(red1.items(), columns = ['Words', 'Count'])

In [28]:
#Input file path 
path='/content/Input.txt'

#Call to the main MapReduce function
output = MapReduce(path)

#Generate the output csv file
output.to_csv('Output.csv')